# DSCI: 525 Milestone 1 - Group 8

### Imports

In [20]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage
import dask.dataframe as dd

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

C:\Users\SantiagoRugelesSch\miniconda3\envs\525\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


### Downloading the data

In [3]:
%pwd

'C:\\Users\\SantiagoRugelesSch\\DSCI_525\\525-group8\\notebooks'

In [4]:
%cd ~/DSCI_525/525-group8

C:\Users\SantiagoRugelesSch\DSCI_525\525-group8


In [13]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "dailyrainfall/" #New directory to include the data

In [14]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # loading all articles
files = data["files"]             # file data available ar the specific URL witht he article ID
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

In [15]:
%%time
files_to_dl = ["data.zip"]  # Grab only the data.zip 
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"]) # Retrieve the data using the figshare API

Wall time: 2min 36s


In [16]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory) #Extract CSVs to output rainfall directory

Wall time: 1min 27s


### Combining data CSVs

In [5]:
#Test to check the reding command
use_cols = ["time", "lat_min", "lat_max", "lon_min","lon_max","rain (mm/day)"]
BCC_df = pd.read_csv("dailyrainfall/BCC-CSM2-MR_daily_rainfall_NSW.csv", usecols=use_cols, parse_dates=["time"])
BCC_df

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-35.887649,-34.766162,141.1875,142.3125,4.265520e-17
1,1889-01-02 12:00:00,-35.887649,-34.766162,141.1875,142.3125,2.728501e-20
2,1889-01-03 12:00:00,-35.887649,-34.766162,141.1875,142.3125,5.221255e-24
3,1889-01-04 12:00:00,-35.887649,-34.766162,141.1875,142.3125,2.869350e-17
4,1889-01-05 12:00:00,-35.887649,-34.766162,141.1875,142.3125,2.290091e-15
...,...,...,...,...,...,...
3035335,2014-12-27 12:00:00,-30.280211,-29.158723,152.4375,153.5625,1.218327e+01
3035336,2014-12-28 12:00:00,-30.280211,-29.158723,152.4375,153.5625,6.209972e-01
3035337,2014-12-29 12:00:00,-30.280211,-29.158723,152.4375,153.5625,3.562820e-04
3035338,2014-12-30 12:00:00,-30.280211,-29.158723,152.4375,153.5625,4.175968e+01


In [13]:
%%time
%memit
# Join all data together
#
import pandas as pd
use_cols = ["time", "lat_min", "lat_max", "lon_min","lon_max","rain (mm/day)"]
files = glob.glob('dailyrainfall/*.csv')
df = pd.DataFrame(columns = use_cols)
for file in files:
    if len(re.findall(r'NSW', file)) > 0: # Used to ignore rainfal SYD as it misses most columns
        print(file)
        df2 = (pd.read_csv(file,usecols=use_cols, parse_dates=["time"]).
               assign(model=re.findall(r'\\(.+?)_', file)[0])) # Create model column
        df = pd.concat([df, df2], axis=0)



peak memory: 1245.37 MiB, increment: 0.61 MiB
dailyrainfall\ACCESS-CM2_daily_rainfall_NSW.csv
dailyrainfall\ACCESS-ESM1-5_daily_rainfall_NSW.csv
dailyrainfall\AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
dailyrainfall\BCC-CSM2-MR_daily_rainfall_NSW.csv
dailyrainfall\BCC-ESM1_daily_rainfall_NSW.csv
dailyrainfall\CanESM5_daily_rainfall_NSW.csv
dailyrainfall\CMCC-CM2-HR4_daily_rainfall_NSW.csv
dailyrainfall\CMCC-CM2-SR5_daily_rainfall_NSW.csv
dailyrainfall\CMCC-ESM2_daily_rainfall_NSW.csv
dailyrainfall\EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
dailyrainfall\FGOALS-f3-L_daily_rainfall_NSW.csv
dailyrainfall\FGOALS-g3_daily_rainfall_NSW.csv
dailyrainfall\GFDL-CM4_daily_rainfall_NSW.csv
dailyrainfall\GFDL-ESM4_daily_rainfall_NSW.csv
dailyrainfall\INM-CM4-8_daily_rainfall_NSW.csv
dailyrainfall\INM-CM5-0_daily_rainfall_NSW.csv
dailyrainfall\KIOST-ESM_daily_rainfall_NSW.csv
dailyrainfall\MIROC6_daily_rainfall_NSW.csv
dailyrainfall\MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
dailyrainfall\MPI-ESM1-2-HR_daily_r

In [14]:
%%time
%memit
df.to_csv("dailyrainfall/combined_rainfall_data.csv")
#Create combined data csv in rainfall directory

peak memory: 1714.43 MiB, increment: 1.20 MiB
Wall time: 16min 13s


### Load the combined CSV to memory and perform a simple EDA

In [22]:
%%time
%memit
rainfall_df = pd.read_csv("dailyrainfall/combined_rainfall_data.csv", index_col=0, parse_dates=["time"])
rainfall_df.sample(15)

peak memory: 803.83 MiB, increment: 0.03 MiB


C:\Users\SantiagoRugelesSch\miniconda3\envs\525\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Wall time: 6min 4s


,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
1217985,1949-12-12 12:00:00,-33.926702,-32.984293,145.62500,146.87500,4.219754e+00,TaiESM1
1563431,2014-05-17 12:00:00,-32.984293,-32.041885,140.62500,141.87500,1.238719e-01,NorESM2-MM
3968409,1918-04-09 12:00:00,-30.857027,-29.921967,142.96875,143.90625,1.292285e-13,MPI-ESM1-2-HR
2888689,1986-01-22 12:00:00,-30.280211,-29.158723,149.06250,150.18750,7.376075e-03,EC-Earth3-Veg-LR
282398,1906-09-11 12:00:00,-36.455696,-34.430380,153.00000,155.00000,5.726764e+00,FGOALS-g3
2973488,1966-03-27 12:00:00,-30.280211,-29.158723,151.31250,152.43750,1.128495e+00,EC-Earth3-Veg-LR
50987,1902-09-10 12:00:00,-36.000000,-34.500000,143.00000,145.00000,1.121848e+01,INM-CM4-8
1506950,1982-12-10 12:00:00,-33.644675,-32.523187,152.43750,153.56250,1.276970e-01,EC-Earth3-Veg-LR
844597,1934-12-19 12:00:00,-34.869110,-33.926702,149.37500,150.62500,1.465696e+00,CMCC-CM2-SR5
270290,1999-07-10 12:00:00,-36.000000,-34.500000,151.00000,153.00000,1.167229e+01,INM-CM5-0


In [24]:
%%time
%memit
rainfall_dd = dd.read_csv("dailyrainfall/combined_rainfall_data.csv")

peak memory: 2161.27 MiB, increment: 0.02 MiB
Wall time: 9.42 s


In [28]:
%%time
rainfall_df.head(15)

Wall time: 0 ns


,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2
5,1889-01-06 12:00:00,-36.25,-35.0,140.625,142.5,3.292260e-02,ACCESS-CM2
6,1889-01-07 12:00:00,-36.25,-35.0,140.625,142.5,8.906468e-02,ACCESS-CM2
7,1889-01-08 12:00:00,-36.25,-35.0,140.625,142.5,3.158012e-02,ACCESS-CM2
8,1889-01-09 12:00:00,-36.25,-35.0,140.625,142.5,3.107391e-02,ACCESS-CM2
9,1889-01-10 12:00:00,-36.25,-35.0,140.625,142.5,3.304103e-02,ACCESS-CM2


In [29]:
%%time
%memit
rainfall_df.shape

peak memory: 2341.03 MiB, increment: 0.02 MiB
Wall time: 8.07 s


(62467843, 7)

In [32]:
%%time
%memit
rainfall_df.describe()

peak memory: 1724.11 MiB, increment: 0.09 MiB
Wall time: 53.3 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310482e+01,-3.197757e+01,1.469059e+02,1.482150e+02,1.901170e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204188e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [33]:
rainfall_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62467843 entries, 0 to 3541229
Data columns (total 7 columns):
 #   Column         Dtype         
---  ------         -----         
 0   time           datetime64[ns]
 1   lat_min        float64       
 2   lat_max        float64       
 3   lon_min        float64       
 4   lon_max        float64       
 5   rain (mm/day)  float64       
 6   model          object        
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 3.7+ GB


In [34]:
rainfall_df["model"].value_counts()

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
TaiESM1             3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
AWI-ESM-1-1-LR       966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-LR        966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64

In [35]:
rainfall_df.groupby("model").mean()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
model,,,,,
ACCESS-CM2,-33.125000,-31.875000,146.250000,148.125000,1.787025
ACCESS-ESM1-5,-33.125000,-31.875000,147.187500,149.062500,2.217501
AWI-ESM-1-1-LR,-33.574618,-31.709369,147.187500,149.062500,2.026071
BCC-CSM2-MR,-33.083931,-31.962443,146.812500,147.937500,1.951832
BCC-ESM1,-33.487230,-30.696650,146.250000,149.062500,1.811032
CMCC-CM2-HR4,-32.984293,-32.041885,146.875000,148.125000,2.279350
CMCC-CM2-SR5,-32.984293,-32.041885,146.875000,148.125000,2.383389
CMCC-ESM2,-32.984293,-32.041885,146.875000,148.125000,2.266125
CanESM5,-33.490987,-30.700020,146.250000,149.062500,1.894328


In [36]:
rainfall_df.memory_usage()

Index            499742744
time             499742744
lat_min          499742744
lat_max          499742744
lon_min          499742744
lon_max          499742744
rain (mm/day)    499742744
model            499742744
dtype: int64

In [39]:
## Change dtypes for storage 
rainfall_df.dtypes

time             datetime64[ns]
lat_min                 float64
lat_max                 float64
lon_min                 float64
lon_max                 float64
rain (mm/day)           float64
model                    object
dtype: object

In [42]:
rainfall_df_2 = rainfall_df.astype({'lat_min':'float32', 
                                  'lat_max':'float32',
                                    'lon_min': 'float32',
                                    'lon_max' : 'float32',
                                    'model' : 'str'
                                    })

In [43]:
rainfall_df_2.dtypes

time             datetime64[ns]
lat_min                 float32
lat_max                 float32
lon_min                 float32
lon_max                 float32
rain (mm/day)           float64
model                    object
dtype: object

### Perform a simple EDA in R

In [6]:
# Rachel and Rui